<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Popular MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [93]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Popular
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])
ratings.drop(['timestamp'],axis=1)
ratings.head(5)

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Genres Matrix

In [94]:
m_cols = ['movie id','movie title','release date','video release date','IMDb URL',
          'unknown','Action','Adventure','Animation','Children','Comedy','Crime',
          'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
          'Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 100k\\u.item', sep='|', names=m_cols,encoding='latin-1')
movies = movies.set_index('movie id')
movies = movies.drop(['video release date'],axis = 1)
genres = movies.drop(['movie title','release date','IMDb URL'],axis = 1)
genres

,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


# Number of genres per movie and Document Frequency

In [95]:
num_genres = genres.sum(axis=1)
df = genres.sum(axis = 0)
df = pd.DataFrame(df)
df.index.name = "Attribute"
df.columns = ['Document Frequency']
df

,Document Frequency
Attribute,
unknown,2
Action,251
Adventure,135
Animation,42
Children,122
Comedy,505
Crime,109
Documentary,50
Drama,725


# Normalize Genre Matrix

In [96]:
genres.fillna(0)
normalized = genres.divide(((((genres.T)**2).sum())**(1/2)),axis=0)
normalized

Attribute,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,
1,0.0,0.00000,0.00000,0.57735,0.57735,0.57735,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.57735,0.57735,0.00000,0.00000,0.00000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.577350,0.0,0.0
3,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0
4,0.0,0.57735,0.00000,0.00000,0.00000,0.57735,0.00000,0.0,0.577350,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.57735,0.0,0.577350,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.577350,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1679,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.707107,0.0,0.0
1680,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.707107,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.000000,0.0,0.0


# Unpivot Ratings Matrix

In [97]:
ratings = ratings.pivot(index = 'user',columns='item',values ='rating')
count = ratings.count(axis=1)
ratings = ratings.fillna(0)
ratings

item,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Profile matrix

In [98]:
profile = pd.DataFrame((ratings).values.dot(normalized.values), columns=normalized.columns, index=(ratings).index)
profile

Attribute,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,
1,4.0,146.915413,67.220282,26.015019,33.109273,234.707756,53.753667,24.0,328.270336,3.809401,3.535534,28.476459,21.872534,11.692388,109.110960,102.916993,119.910473,52.743606,14.731686
2,0.0,22.536304,7.064495,2.309401,7.576986,49.951537,19.720732,0.0,104.753197,1.732051,4.809401,3.621320,2.121320,7.621320,43.552990,8.089439,26.542877,5.959683,0.000000
3,0.0,22.288125,7.849332,0.000000,0.000000,24.752866,17.712296,5.0,47.854431,0.000000,2.732051,8.223615,2.568914,21.137464,9.686196,12.158733,32.293953,7.771982,0.000000
4,0.0,17.279674,7.357388,0.000000,0.000000,16.422285,12.137464,5.0,20.041087,0.000000,0.000000,2.828427,2.886751,13.118802,7.271602,12.892922,28.968045,4.236068,0.000000
5,4.0,105.038789,58.341967,31.078522,43.473999,184.271504,19.509586,0.0,50.075682,2.500000,3.535534,57.421899,23.662149,4.886751,23.858328,66.677337,31.879091,24.005723,2.886751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,45.160950,19.846450,1.788854,1.788854,52.794951,8.154336,0.0,63.421735,2.000000,0.000000,3.000000,1.788854,2.309401,33.112698,18.902895,32.729347,9.309037,0.000000
940,0.0,44.211123,19.020119,4.809401,12.256630,111.196662,19.995507,0.0,129.288974,0.000000,7.656854,9.654701,13.516143,7.064495,51.328315,33.955148,40.841576,28.574314,0.000000
941,0.0,23.553176,14.601908,7.562357,4.675606,20.562357,1.732051,0.0,17.192388,0.000000,0.000000,0.000000,1.788854,3.535534,2.236068,18.469908,17.049159,2.236068,0.000000


# Compute Predictions (Genres . Profile^T)

In [99]:
x = (normalized).divide((df.T).values)
prediction = pd.DataFrame(x.values.dot((profile.T).values), columns=(profile.T).columns, index=x.index)
prediction = prediction.T
prediction

movie id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,0.782633,0.901232,0.477731,0.867685,0.821957,0.452787,1.040696,0.686436,0.452787,0.845455,...,0.751691,0.452787,0.452787,0.452787,0.452787,0.452787,0.650168,0.632529,0.464768,0.452787
2,0.124711,0.143104,0.105749,0.192366,0.248930,0.144487,0.158802,0.176385,0.144487,0.161522,...,0.138264,0.144487,0.144487,0.144487,0.144487,0.144487,0.199458,0.226851,0.098914,0.144487
3,0.028299,0.159119,0.128661,0.117675,0.206210,0.066006,0.131797,0.066408,0.066006,0.124076,...,0.153766,0.066006,0.066006,0.066006,0.066006,0.066006,0.118707,0.074403,0.049016,0.066006
4,0.018775,0.137844,0.115411,0.074481,0.146882,0.027643,0.109811,0.034735,0.027643,0.061735,...,0.130287,0.027643,0.027643,0.027643,0.027643,0.027643,0.102425,0.040363,0.032519,0.027643
5,0.843626,0.564448,0.127008,0.492159,0.216544,0.069070,0.515652,0.456285,0.069070,0.287919,...,0.385719,0.069070,0.069070,0.069070,0.069070,0.069070,0.158110,0.117141,0.364894,0.069070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.093415,0.264040,0.130396,0.214744,0.168981,0.087478,0.194197,0.119330,0.087478,0.154567,...,0.219429,0.087478,0.087478,0.087478,0.087478,0.087478,0.186998,0.156651,0.104544,0.087478
940,0.251243,0.276981,0.162715,0.331781,0.302814,0.178330,0.363820,0.288089,0.178330,0.410677,...,0.239607,0.178330,0.178330,0.178330,0.178330,0.178330,0.261999,0.273040,0.220191,0.178330
941,0.149590,0.155841,0.067925,0.091376,0.062082,0.023714,0.146077,0.059326,0.023714,0.039038,...,0.114383,0.023714,0.023714,0.023714,0.023714,0.023714,0.054432,0.023169,0.040718,0.023714


In [107]:
prediction.min().min()

0.0

In [108]:
OldMin = 0
OldMax = 3
NewMin = 1
NewMax = 5
NewValue = (((prediction - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin
NewValue

movie id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,2.043511,2.201642,1.636975,2.156914,2.095942,1.603716,2.387595,1.915248,1.603716,2.127274,...,2.002254,1.603716,1.603716,1.603716,1.603716,1.603716,1.866890,1.843372,1.619690,1.603716
2,1.166282,1.190806,1.140998,1.256488,1.331907,1.192650,1.211737,1.235180,1.192650,1.215362,...,1.184352,1.192650,1.192650,1.192650,1.192650,1.192650,1.265944,1.302467,1.131885,1.192650
3,1.037732,1.212158,1.171548,1.156900,1.274946,1.088008,1.175730,1.088544,1.088008,1.165435,...,1.205022,1.088008,1.088008,1.088008,1.088008,1.088008,1.158276,1.099204,1.065354,1.088008
4,1.025033,1.183792,1.153881,1.099309,1.195842,1.036857,1.146414,1.046313,1.036857,1.082313,...,1.173716,1.036857,1.036857,1.036857,1.036857,1.036857,1.136566,1.053818,1.043359,1.036857
5,2.124835,1.752597,1.169344,1.656213,1.288725,1.092093,1.687536,1.608380,1.092093,1.383892,...,1.514292,1.092093,1.092093,1.092093,1.092093,1.092093,1.210813,1.156188,1.486525,1.092093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,1.124553,1.352053,1.173861,1.286325,1.225309,1.116638,1.258929,1.159107,1.116638,1.206090,...,1.292572,1.116638,1.116638,1.116638,1.116638,1.116638,1.249331,1.208868,1.139393,1.116638
940,1.334990,1.369308,1.216954,1.442374,1.403753,1.237773,1.485093,1.384119,1.237773,1.547569,...,1.319476,1.237773,1.237773,1.237773,1.237773,1.237773,1.349332,1.364053,1.293589,1.237773
941,1.199454,1.207788,1.090567,1.121835,1.082776,1.031618,1.194769,1.079101,1.031618,1.052050,...,1.152511,1.031618,1.031618,1.031618,1.031618,1.031618,1.072575,1.030893,1.054290,1.031618


In [110]:
NewValue.min().min()

1.0